In [2]:
import json
import pandas as pd

In [3]:
file_path = './batch_국방부훈령_output.jsonl' ## 배치로 나온 ouptut 결과
raw_data_path = './국방부훈령_JSONL_RESULT.jsonl' ## 배치 전 데이터

## josnl 파일 불러오기
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            data.append(json.loads(line.strip()))
    return data

In [5]:
## 배치 결과에서 답변만 뽑아내는 법
def extract_content(data):
    contents = []
    for item in data:
        if 'response' in item and 'body' in item['response'] and 'choices' in item['response']['body']:
            for choice in item['response']['body']['choices']:
                if 'message' in choice and 'content' in choice['message']:
                    contents.append(choice['message']['content'])
    return contents

## 배치 입력에서 user content만 뽑아내기
def extract_user_content(data):
    user_contents = []
    for item in data:
        if 'body' in item and 'messages' in item['body']:
            for message in item['body']['messages']:
                if message['role'] == 'user' and 'content' in message:
                    user_contents.append(message['content'])
    return user_contents

In [38]:
jsonl_data = load_jsonl(file_path)
raw_data = pd.read_excel('./국방부훈령_batch_input.xlsx')[['document','user']]
contents_data = extract_content(jsonl_data)
# user_prompt = extract_user_content(raw_data)

In [8]:
# data = pd.DataFrame({
#     'user_promt' : user_prompt,
#     'assistant' : contents_data
# })

In [39]:
raw_data['contents'] = contents_data
raw_data

,document,user,contents
0,3군 공통군수지원 훈령,제1장 총 칙 제1조(목적) 이 훈령은 육군․해군․공군 상호간의 전시 및 평시 군...,- 이 훈령의 목적이 뭐야? | 이 훈령의 목적에 대해 질문하셨군요. 이 훈령은 육...
1,3군 공통군수지원 훈령,"제1장 총 칙 제2조(적용범위) 이 훈령은 국방부, 육군․해군․공군<이하 “3군”...",- 이 훈령은 어디에 적용되나요? | 이 훈령의 적용 범위에 대한 질문을 주셨군요....
2,3군 공통군수지원 훈령,제1장 총 칙 제3조(정의) 이 훈령에서 사용하는 용어의 정의는 다음 각 호와 같...,"1. ""3군 공통군수지원""이 뭐야? | ""3군 공통군수지원""에 대한 질문을 주셨군요..."
3,3군 공통군수지원 훈령,제1장 총 칙 제4조(지원구분) 3군 공통군수지원은 통합지원과 상호지원으로 구분하...,- 3군 공통군수지원이 뭐야? | 3군 공통군수지원에 대한 질문을 주셨군요. 3군 ...
4,3군 공통군수지원 훈령,제1장 총 칙 제5조(3군 공통군수지원 품목 선정기관) ① 3군 공통군수지원 품목...,- 3군 공통군수지원 품목이 뭐야? | 3군 공통군수지원 품목에 대해 질문하셨군요....
...,...,...,...
8503,국방군사시설 사업에 관한 법률에 따른 승인 등에 관한 훈령,제5장 준공검사 제20조(준공검사 후속조치) ① 국방시설본부장은 준공검사 합격 결...,- 준공검사서 발급은 언제 이루어지나요? | 준공검사서 발급 시점에 대해 질문 주셨...
8504,국방군사시설 사업에 관한 법률에 따른 승인 등에 관한 훈령,제6장 보 칙 제22조(건축승인 및 준공검사 업무의 지도감독) ① 국방시설본부장은...,- 국방시설본부장은 언제 보고를 해야 합니까? | 국방시설본부장의 보고 시기에 대해...
8505,국방군사시설 사업에 관한 법률에 따른 승인 등에 관한 훈령,제6장 보 칙 제23조(사업시행자와의 협의) 국방부장관(군사시설기획관)은 효율적이...,1. 국방부장관이 사업시행자와 협의할 때 어떤 법 조항을 준용해야 합니까? | 국방...
8506,국방군사시설 사업에 관한 법률에 따른 승인 등에 관한 훈령,제6장 보 칙 제24조(기존의 국방·군사시설에 대한 적합확인) 국방시설본부장은 사...,1. 국방시설본부장이 적합확인 신청을 받으면 어떻게 해야 하나요? | 국방시설본부장...


In [46]:
# user_content를 "\n"을 기준으로 분리하고, 각 분리된 값을 새로운 행으로 추가
expanded_rows = []

for index, row in raw_data.iterrows():
    assistant = row['contents'].split('\n')
    for content in assistant:
        expanded_rows.append({'document': row['document'],'reference': row['user'], 'assistant': content})


expanded_df = pd.DataFrame(expanded_rows)

In [47]:
expanded_df

,document,reference,assistant
0,3군 공통군수지원 훈령,제1장 총 칙 제1조(목적) 이 훈령은 육군․해군․공군 상호간의 전시 및 평시 군...,- 이 훈령의 목적이 뭐야? | 이 훈령의 목적에 대해 질문하셨군요. 이 훈령은 육...
1,3군 공통군수지원 훈령,제1장 총 칙 제1조(목적) 이 훈령은 육군․해군․공군 상호간의 전시 및 평시 군...,- 이 훈령은 어떤 상황에서 적용되나요? | 이 훈령의 적용 상황에 대해 궁금해 하...
2,3군 공통군수지원 훈령,제1장 총 칙 제1조(목적) 이 훈령은 육군․해군․공군 상호간의 전시 및 평시 군...,- 군수지원이 뭡니까? | 군수지원에 대해 질문하셨군요. 군수지원은 군대의 작전 수...
3,3군 공통군수지원 훈령,제1장 총 칙 제1조(목적) 이 훈령은 육군․해군․공군 상호간의 전시 및 평시 군...,- 이 훈령은 어떤 군대에 적용되나요? | 이 훈령의 적용 대상에 대해 궁금해 하셨...
4,3군 공통군수지원 훈령,제1장 총 칙 제1조(목적) 이 훈령은 육군․해군․공군 상호간의 전시 및 평시 군...,- 이 훈령의 목적은 무엇을 효율적으로 수행하기 위함인가요? | 이 훈령의 목적에 ...
...,...,...,...
56852,국방군사시설 사업에 관한 법률에 따른 승인 등에 관한 훈령,제6장 보 칙 제25조(재검토기한) 「훈령․예규 등의 발령 및 관리에 관한 규정」...,- 재검토기한이 뭐야? | 재검토기한에 대한 질문을 주셨군요. 재검토기한은 「훈령․...
56853,국방군사시설 사업에 관한 법률에 따른 승인 등에 관한 훈령,제6장 보 칙 제25조(재검토기한) 「훈령․예규 등의 발령 및 관리에 관한 규정」...,- 3년마다 검토해야 하는 이유가 뭡니까? | 3년마다 검토해야 하는 이유에 대해 ...
56854,국방군사시설 사업에 관한 법률에 따른 승인 등에 관한 훈령,제6장 보 칙 제25조(재검토기한) 「훈령․예규 등의 발령 및 관리에 관한 규정」...,- 2022년 1월 1일 기준으로 매 3년이 되는 시점이 언제야? | 2022년 1...
56855,국방군사시설 사업에 관한 법률에 따른 승인 등에 관한 훈령,제6장 보 칙 제25조(재검토기한) 「훈령․예규 등의 발령 및 관리에 관한 규정」...,- 타당성 검토는 누가 하는 거야? | 타당성 검토 주체에 대해 궁금해 하셨군요. ...


In [48]:
## | 구분자로 question, answer 삭제
expanded_df[['question', 'answer']] = expanded_df['assistant'].str.split('|', expand=True)
del expanded_df['assistant']

## 앞에 나열하는 번호 삭제
expanded_df['question'] = expanded_df['question'].str.replace(r'^- ','',regex = True)
expanded_df['question'] = expanded_df['question'].str.replace(r'^(\d\. )','',regex = True)

In [49]:
expanded_df.columns = ['document','reference','question', 'answer']

In [51]:
expanded_df.to_excel('./국방부훈령_질문 증강.xlsx', index = False)